In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [10]:
#Load the csv exported in Part I to a DataFrame
intial_df = pd.read_csv("output_data/cities.csv")
intial_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [30]:
# Humidity Heatmap
#Configure gmaps.
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight.
locations = intial_df[["Lat","Lng"]]
weight = intial_df["Humidity"]
#Plot 1st map
gmap = gmaps.figure(center = [0,0], zoom_level=2)


#Add Heatmap layer to map.
hmlayer = gmaps.heatmap_layer(locations,weight,
                             dissipating=False, max_intensity=200,
                               point_radius = 5)
gmap.add_layer(hmlayer)
gmap

Figure(layout=FigureLayout(height='420px'))

In [4]:
#Create new DataFrame fitting weather criteria

#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.

In [5]:
# Hotel Map

#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure